In [4]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf 
import os
import requests

from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization
from sklearn import metrics
from bs4 import BeautifulSoup

In [5]:
def load_data(phish_dir, not_phish_dir):
    phish_files = [f for f in os.listdir(phish_dir) if f.endswith('.html')]
    not_phish_files = [f for f in os.listdir(not_phish_dir) if f.endswith('.html')]

    phish_texts = []
    for f in phish_files:
        with open(os.path.join(phish_dir, f), 'r', encoding='latin-1') as file:
            soup = BeautifulSoup(file, 'html.parser')
            text = soup.get_text()
            phish_texts.append(text)

    not_phish_texts = []
    for f in not_phish_files:
        with open(os.path.join(not_phish_dir, f), 'r', encoding='latin-1') as file:
            soup = BeautifulSoup(file, 'html.parser')
            text = soup.get_text()
            not_phish_texts.append(text)

    return phish_texts, not_phish_texts


In [6]:
phish_texts, not_phish_texts = load_data('../datasets/phishing-web/Phish', '../datasets/phishing-web/NotPhish')

labels = [1] * len(phish_texts) + [0] * len(not_phish_texts)

df = pd.DataFrame({"Sentence": phish_texts + not_phish_texts, "Label": labels})

In [7]:
sentences=df['Sentence'].values

In [8]:
def convert_to_ascii(sentence):
    sentence_ascii = []

    for i in sentence:
        if ord(i) < 8222:
            sentence_ascii.append(134)
        if ord(i) == 8221: 
            sentence_ascii.append(129)
        if ord(i) == 8220: 
            sentence_ascii.append(130)
        if ord(i) == 8216:
            sentence_ascii.append(131)
        if ord(i) == 8217: 
            sentence_ascii.append(132)
        if ord(i) == 8211: 
            sentence_ascii.append(133)

        if ord(i) <= 128:
            sentence_ascii.append(ord(i))

    if len(sentence_ascii) > 10000:
        sentence_ascii = sentence_ascii[:10000]
    else:
        sentence_ascii += [0] * (10000 - len(sentence_ascii))
    
    zer = np.array(sentence_ascii).reshape((100, 100))
    return zer


In [9]:
arr=np.zeros((len(sentences),100,100))

for i in range(len(sentences)):
    
    image=convert_to_ascii(sentences[i])

    x=np.asarray(image,dtype='float')
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    image/=128

    arr[i]=image

In [10]:
data = arr.reshape(arr.shape[0], 100, 100, 1)
data.shape

(10355, 100, 100, 1)

In [11]:
y=df['Label'].values

trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)
trainX, validX, trainY, validY = train_test_split(trainX,trainY, test_size=0.2, random_state=42)

In [12]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [13]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 256)       0

In [14]:
batch_size = 128
num_epoch = 10

mc = ModelCheckpoint('../models/phishing-web.h5', save_best_only=True, save_weights_only=True)
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( validX,  validY),
          callbacks=[mc]
                     )

Epoch 1/10
52/52 [==============================] - 45s 865ms/step - loss: 0.4822 - accuracy: 0.8016 - val_loss: 0.4467 - val_accuracy: 0.8256
Epoch 2/10
52/52 [==============================] - 44s 851ms/step - loss: 0.3979 - accuracy: 0.8354 - val_loss: 0.4067 - val_accuracy: 0.8328
Epoch 3/10
52/52 [==============================] - 45s 857ms/step - loss: 0.3801 - accuracy: 0.8482 - val_loss: 0.4026 - val_accuracy: 0.8358
Epoch 4/10
52/52 [==============================] - 44s 856ms/step - loss: 0.3627 - accuracy: 0.8545 - val_loss: 0.4072 - val_accuracy: 0.8352
Epoch 5/10
52/52 [==============================] - 45s 859ms/step - loss: 0.3483 - accuracy: 0.8592 - val_loss: 0.4526 - val_accuracy: 0.8171
Epoch 6/10
52/52 [==============================] - 45s 864ms/step - loss: 0.3359 - accuracy: 0.8628 - val_loss: 0.3673 - val_accuracy: 0.8558
Epoch 7/10
52/52 [==============================] - 45s 861ms/step - loss: 0.3034 - accuracy: 0.8837 - val_loss: 0.3693 - val_accuracy: 0.8515

In [15]:
model.load_weights('../models/phishing-web.h5')

y_prob = model.predict(testX)
threshold = 0.5
y_pred = (y_prob > threshold).astype(int)

clf_report = metrics.classification_report(testY, y_pred, digits=4)
cnf_matrix = metrics.confusion_matrix(testY, y_pred)
TN, FP, FN, TP = cnf_matrix.ravel()
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)

other_metrics = pd.DataFrame({'TPR': '%.4f' % TPR, 
                              'FPR': '%.4f' % FPR, 
                              'FNR': '%.4f' % FNR
                              }, index=[0]).to_string(col_space=9, index=False)
print(clf_report)
print(cnf_matrix)
print(other_metrics)

65/65 [==============================] - 4s 59ms/step
              precision    recall  f1-score   support

           0     0.9036    0.8879    0.8957      1204
           1     0.8480    0.8685    0.8581       867

    accuracy                         0.8798      2071
   macro avg     0.8758    0.8782    0.8769      2071
weighted avg     0.8803    0.8798    0.8800      2071

[[1069  135]
 [ 114  753]]
      TPR       FPR       FNR
   0.8685    0.1121    0.1315


In [56]:
def phishing_web_predict(model, sentence):
    preprocessed_sentence = convert_to_ascii(sentence)
    x = np.asarray(preprocessed_sentence, dtype='float')
    x = cv2.resize(x, dsize=(100, 100), interpolation=cv2.INTER_CUBIC)
    x /= 128
    x = np.expand_dims(x, axis=0) 
    x = np.expand_dims(x, axis=-1) 
    
    prediction = model.predict(x)
    
    return prediction[0][0]

def get_html_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print("Error while getting HTML", e)
        return None

test_url = "https://google.com"

your_sentence = get_html_from_url(test_url)
prediction = phishing_web_predict(model, your_sentence)
if prediction > 0.5:
    print("Prediction: 1")
else:
    print("Prediction: 0")


1/1 [==============================] - 0s 17ms/step
Prediction: 0
